Install the prerequisites:

In [1]:
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00


Login into HuggingFace, so the program can access the stored models. The Llama series requires additional registration with Meta:

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Do the imports:

In [3]:
import torch
from transformers import AutoTokenizer, pipeline

Define the model identifier:

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"

Load the tokenizer:

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [6]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False)}, clean_up_tokenization_spaces=False)


Instantiate text generation pipeline:

In [7]:
text_gen_pipeline = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
print(text_gen_pipeline.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [10]:
sequences = text_gen_pipeline(
    text_inputs='What is the answer to the ultimate question of life, the universe, and everything?',
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=2,
    early_stopping=True
)
for s in sequences:
    print(f"RESULT: {s['generated_text']}")


RESULT: What is the answer to the ultimate question of life, the universe, and everything?

Answer: The answer to the ultimate question of life, the universe, and everything is 42.

Explanation:

The answer 42 is a humorous and satirical response to the idea that there is a single, definitive answer to the ultimate questions of life, the universe, and everything. The number 42 is often used as a placeholder or a nonsensical answer in situations where a more meaningful or definitive
RESULT: What is the answer to the ultimate question of life, the universe, and everything?

Answer: The answer to the ultimate question of life, the universe, and everything is 42.

Explanation:

The answer 42 is a humorous and satirical response to the idea that there is a single, definitive answer to the ultimate questions of life, the universe, and everything. The answer was first proposed by Douglas Adams in his science fiction series "The Hitchhiker's Guide to the Galaxy," where


In [ ]:
print(sequences)

[{'generated_text': 'What is the answer to the ultimate question of life, the universe, and everything?\n\nAnswer: The answer to the ultimate question of life, the universe, and everything is 42.\n\nExplanation:\n\nThe answer 42 is a humorous and satirical response to the idea that there is a single, definitive answer to the ultimate questions of life, the universe, and everything. The answer was first proposed by Douglas Adams in his science fiction series "The Hitchhiker\'s Guide to the Galaxy," where it is revealed as the result of a supercomputer named Deep Thought\'s calculation of the answer to the ultimate question.\nIn the series, the character Arthur Dent discovers that the answer 42 is not only the ultimate answer to life, the universe, and everything, but also the only thing that the supercomputer can ever compute. The character is left to ponder'}, {'generated_text': 'What is the answer to the ultimate question of life, the universe, and everything?\n\nComment: The answer t